In [455]:
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px 
import json
import csv
from dash import dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc

In [456]:
df=pd.read_csv('datatran.2020.csv', sep=';') #leitura do dataframe em csv.
data_list = df.values.tolist()

In [457]:
def separar_coluna(lista, num_coluna):                                                      #Argumentos sao a tabela em formato de lista
                                                                                               # e o numero da coluna que voce quer retirar da tabela
    conteudo_coluna = []                                                                       # Inserindo coluna vazia para receber os valores da coluna desejada

    for i in lista:                                                                         # o i percorre a lista e quando ele chegar no indice desejado 
        conteudo_coluna.append(i[num_coluna])                                                  # ele adiciona na coluna vazia
    
    return conteudo_coluna

In [458]:
def excluir_duplicado(coluna_valor_duplicado):                                                # Funcao para excluir valor duplicado
                                                                                              # Os argumentos sao: 
  lista_valor_unico = []                                                                      #a coluna que possui os valores duplicados

  for i in range (len(coluna_valor_duplicado)):                                               #laco possui o tamanho da coluna que vai ser unificada para verficar todos os valores
                                                                                              # A condicao estabelecida verfica se todos os valores da coluna duplicada
      if coluna_valor_duplicado[i] not in lista_valor_unico:                                  # estao na coluna criada, caso nao esteja, ele adiciona nessa coluna onde nao possui                                                                                              # ele adiciona esse valor
         lista_valor_unico.append(coluna_valor_duplicado[i])                                  #valores repetidos
  
  return lista_valor_unico

In [459]:
def acidente_dia_objeto_estudo(lista, dia_semana, objeto_estudo, num_coluna_dia_semana,num_coluna_objeto_estudo):
    objeto = []
    dia_sem = 0

    for linha in lista:

        if linha[num_coluna_dia_semana] == dia_semana and linha[num_coluna_objeto_estudo] == objeto_estudo:
            dia_sem += 1
    
    objeto.append(dia_semana)
    objeto.append(dia_sem)
    objeto.append(objeto_estudo)
    
    return objeto

In [460]:
def lista_completa (objeto_single,lista, dia_semana,num_coluna_dia_semana,num_coluna_objeto_estudo):
   
    lista_comp = []

    for i in objeto_single:
        
        lista_comp.append(acidente_dia_objeto_estudo(lista,dia_semana,i,num_coluna_dia_semana,num_coluna_objeto_estudo))
    
    return lista_comp

In [461]:
def calculo_mortes(data_list, uf): 
    mortos_estado = [] 
    for linha in data_list:
        if linha[4] == uf:  
            mortos_estado.append(linha[18]) 


    return sum(mortos_estado) 




In [462]:
def calculo_feridos(data_list, uf): 
    feridos_estado = [] 
    for linha in data_list:
        if linha[4] == uf:    
            feridos_estado.append(linha[23])


    return sum(feridos_estado) 



In [463]:
def estado(data_list, num_coluna):
    lista2 = []
    lista_estados =[]
    for i in data_list:
        lista2.append(i[num_coluna])
    for i in lista2:
        if i not in lista_estados:
            lista_estados.append(i)
    lista_estados.sort()

    return lista_estados

In [464]:
def filtro_periodo(p):
    lista = []
    for i in nnova_df:
        if p == i[2]:
            lista.append(i)
        
    return lista

# GRAFICO DE ROSCA

# - Filtrando as tabelas para o Grafico de Rosca

In [465]:
meses=pd.read_csv('meses-.csv') #leitura da planilha de meses 
meses_list=meses.values.tolist() #tranformar a base de meses em uma lista

#separar a coluna 'data' da meses_list para comparar com as datas da base de dados (coluna [1] = "data-inversa")

compara_mes = separar_coluna(meses_list, 1)

#separar a coluna de datas (dia/mes/ano).

data_2020 = separar_coluna(data_list, 1)


# separar a coluna de feridos graves
feridos = separar_coluna(data_list, 20)

#substituindo as datas (mes/dia/ano) pelos seus respectivos meses

meses_2020 = [] #lista vazia que ira receber o elementos do laço.

for c in range(0,len(compara_mes)): #percorrer a lista de meses
    
    for d in range(0,len(data_2020)): #percorrer a lista de datas (mes/dia/ano)
        
        if compara_mes[c][0]==data_2020[d][5] and compara_mes[c][1]==data_2020[d][6]: #comparar os algarismos de cada string presente na lista, linha [c] e caractere [0]. ex: compara_mes[c] para c=0 == compara_mes[0] = '1/'; compara_mes[0][0]="1" e compara_[0][1]='/'. o mesmo ocorre com a lista data_2020 (datas da base de dados) que se inicia com o mês. Fazendo então a comparação do primeiro e do segundo caracterie de cada lista, caso o mes tenha um algarismo (janeiro = 1, fevereiro = 2), o segundo algarismo comparado será a barra (1 = 1 ; / = /; 1/ = janeiro), caso o mes tenham 2 algarismos (dezembro=12; novermbro=11), a segunda comparação sera a comparação do segundo algarismo do mes (1 = 1; 2 = 2; 12/ = Dezembro).
            
            meses_2020.append(meses_list[c][0]) #adicionar o mes correspondente à data na lista, de acordo com a comparação feita no laço if

meses_2020 #lista de meses correspondentes a cada data
# separar a coluna de data_list

tipos=[] #criar uma lista vazia para receber a coluna de tipos de acidentes 

for linha in data_list: #percorrer toda a base de dados
   
    tipos.append(linha[9]) #acrescentar à lista somente a coluna [9] == tipo de acidentes

inteiros_feridos=list(map(int, feridos))

total = sum(inteiros_feridos)
#CONTRUÇÃO DO GRÁFICO
#grafico de rosca
# TEMA: quantidade de feridos graves de cada tipo de acidente em função do mês
#lista 'meses_20202' com os meses do ano 
#lista 'tipos' com os tipos de acidentes
#lista 'feridos'  com a quantidade de feridos graves por acidente
                            

# - Construindo o Grafico de Rosca

In [466]:
df=dict(MESES=meses_2020, TIPOS=tipos, FERIDOS=feridos, Total='Total') #dicionario que irá receber os dados das listas que estarão no grafico

fig_sunburst=px.sunburst(df,path=['Total','MESES','TIPOS'], values='FERIDOS', #plotando o gráfico.
                        branchvalues = 'total', #determina como os itens em valores são somados. 
                        color_discrete_sequence= px.colors.sequential.Plasma_r,  #mudar a textura de cor do gráfico 
                        maxdepth=2) #divisões do gráfico

fig_sunburst.update_traces(hovertemplate='<b>%{label}</b> <br> Feridos: %{value}<br>',
                        insidetextorientation='radial', 
                        textfont_color='#aaaaaa', 
                        selector=dict(type='sunburst'))

                    
fig_sunburst.update_layout(plot_bgcolor="#272b30",paper_bgcolor="#272b30",margin=dict(l=50, r=50, t=50, b=50)) #características do layout do gráfico 

C:\Users\danie\AppData\Local\Programs\Python\Python38\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\danie\AppData\Local\Programs\Python\Python38\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\danie\AppData\Local\Programs\Python\Python38\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



# - Plotando o Grafico de Rosca no Dash

In [467]:
app = dash.Dash()
app.layout = html.Div( children=[
    html.H1(children='Feridos por tipo de acidentes', style={'textAlign': 'center',}),  #característicacs de layout na página html,título,sub-título,centralização e estilo da fonte
    html.Div(children=
        'Quantidade de feridos de cada tipo de acidentes por mês de 2020.'
    ,style={'textAlign': 'center',}),
    dcc.Graph(id='sunburst-graph',figure=fig_sunburst) #criando identidade pro gráfico 
])

# GRAFICO DE CAUSAS

In [468]:
#Barra - Causas

dias_semana = separar_coluna(data_list,2)

dias_semana_single = excluir_duplicado(dias_semana)

causas = separar_coluna(data_list,8)

causas_single = excluir_duplicado(causas)


segunda = lista_completa(causas_single,data_list,'segunda', 2, 8)
terca = lista_completa(causas_single,data_list,'terça', 2, 8)
quarta = lista_completa(causas_single,data_list,'quarta', 2, 8)
quinta = lista_completa(causas_single,data_list,'quinta', 2, 8)
sexta = lista_completa(causas_single,data_list,'sexta', 2, 8)
sabado = lista_completa(causas_single,data_list,'sábado', 2, 8)
domingo = lista_completa(causas_single,data_list,'domingo', 2, 8)


dias = segunda + terca + quarta + quinta + sexta + sabado + domingo
colunas = ['Dias da Semana', 'Quantidade', 'Causas']



with open('causas.csv', 'w', encoding='UTF8', newline='') as f:
     writer = csv.writer(f)
     writer.writerow(colunas)
     writer.writerows(dias)


df = pd.read_csv('causas.csv')

                      
def calculo_causas(lista, causa_acidente):
  tipos=[]
  segunda = 0
  terça = 0
  quarta = 0
  quinta = 0
  sexta = 0
  sabado = 0
  domingo = 0
  for linha in lista:
    if linha[2] == 'segunda' and linha[8] == causa_acidente:
      segunda +=1
  for linha in lista:
    if linha[2] == 'terça' and linha[8] == causa_acidente:
      terça +=1
  for linha in lista:
    if linha[2] == 'quarta' and linha[8] == causa_acidente:
      quarta +=1
  for linha in lista:
    if linha[2] == 'quinta' and linha[8] == causa_acidente:
      quinta +=1
  for linha in lista:
    if linha[2] == 'sexta' and linha[8] == causa_acidente:
      sexta +=1
  for linha in lista:
    if linha[2] == 'sábado' and linha[8] == causa_acidente:
      sabado +=1
  for linha in lista:
    if linha[2] == 'domingo' and linha[8] == causa_acidente:
      domingo +=1
  tipos.append(segunda)
  tipos.append(terça)
  tipos.append(quarta)
  tipos.append(quinta)
  tipos.append(sexta)
  tipos.append(sabado)
  tipos.append(domingo)
  return tipos


# GRAFICO DOS ACIDENTES POR PERIODO

In [469]:

dias_semana = separar_coluna(data_list,2)

dias_semana_single = excluir_duplicado(dias_semana)

periodo = separar_coluna(data_list,11)

periodo_single = excluir_duplicado(periodo)


segunda = lista_completa(periodo_single, data_list,'segunda', 2, 11)
terca = lista_completa(periodo_single, data_list,'terça', 2, 11)              
quarta = lista_completa(periodo_single, data_list,'quarta', 2, 11)
quinta = lista_completa(periodo_single, data_list,'quinta', 2, 11)
sexta = lista_completa(periodo_single, data_list,'sexta', 2, 11)
sabado = lista_completa(periodo_single, data_list,'sábado', 2, 11)
domingo = lista_completa(periodo_single, data_list,'domingo', 2, 11)


dias = segunda + terca + quarta + quinta + sexta + sabado + domingo
colunas = ['Dias da Semana', 'Quantidade', 'Periodo']



with open('nova_basedados.csv', 'w', encoding='UTF8', newline='') as f:
     writer = csv.writer(f)
     writer.writerow(colunas)
     writer.writerows(dias)

nova_df = pd.read_csv('nova_basedados.csv')
fig = px.bar(nova_df, x='Dias da Semana', 
                      y = 'Quantidade', 
                      color='Periodo' , 
                      barmode='group')

periodo_single.append('Todos os períodos')

In [470]:
nnova_df = nova_df.values.tolist()

# GRAFICO COROPLETICO

In [471]:
#Mapa Coroplético

newdata = pd.read_csv('newdata.csv') 
brasil_states = json.load(open("brazil_geo.json", "r"))

estados = (estado(data_list,4))            


soma_mortes = []
soma_feridos = []
geral = []
for i in estados:
    mortes = calculo_mortes(data_list, i) 
    feridos = calculo_feridos(data_list, i) 
    total = mortes + feridos 

    geral.append(total) 
    soma_mortes.append(mortes) 
    soma_feridos.append(feridos) 

 
dados = [] 
for i in range (len(geral)):
    dados.append([soma_mortes[i], soma_feridos[i], geral[i], estados[i]]) 

In [472]:

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

app.layout = html.Div(children=[

    html.H1(children='Acidentes rodoviarios no brasil em 2020', style={'textAlign': 'center',}),
    
    #GRAFICO CAUSAS

    html.H2(children ='Causas', style={'textAlign': 'center',}),
        
        html.P(children ='Selecione a causa a ser analisada:', style={'textAlign': 'center',}),
                
                dcc.Dropdown(causas_single, value ='Animais na Pista', id ='selecione_causa'),
                
                dcc.Graph(id = 'my_graph', figure = {}),
                
    #GRAFICO PERIODO

    html.H2(children='Acidentes por periodo', style={'textAlign': 'center',}),
        
        html.P(children ='Selecione o período do dia:', style={'textAlign': 'center',}),
                
                dcc.Dropdown(periodo_single, value='Todos os períodos', id='botao-dropdown'),
                
                dcc.Graph(id='grafico-periodo',figure=fig),

    #GRAFICO ROSCA

    html.H2(children='Feridos por tipo de acidentes', style={'textAlign': 'center',}),                 #característicacs de layout na página html,título,sub-título,centralização e estilo da fonte
        
        html.Div(children=
            
            'Quantidade de feridos de cada tipo de acidentes por mês de 2020.', style={'textAlign': 'center',}),
        
        dcc.Graph(id='sunburst-graph',figure=fig_sunburst),                                             #criando identidade pro gráfico 

    #GRAFICO COROPLETICO

    html.H2('Acidentes rodoviários 2020', style={'textAlign': 'center',}),                              #Tag de Cabeçalho – onde h1 tem um tamanho maior, que vai diminuindo até chegar no h6.
        
        html.P("Selecione o parâmetro", style={'textAlign': 'center',}),                                # Tag de parágrafo
    
    dcc.RadioItems(                                                                                     # o input do tipo "radioitems" se refere a um elemento que possui opções que podem ser marcadas ou desmarcadas, caracterizadas por um pequeno círculo ao lado do texto.
        id='cor',                                                                                       #identificação
        options=[
       {'label': 'Mortos', 'value': 'Mortos'},
       {'label': 'Feridos', 'value': 'Feridos'},
       {'label': 'Total de acidentados', 'value': 'Total'},
   ], #opções a serem marcadas
        value="Total", #correspondência (depende da opção escolhida pelo usuário)
        inline=True, #(opções alinhadas lado a lado)
        style={'textAlign': 'center',}   
    ),


    dcc.Graph(id="mapa"), #Criação de um gráfico

])






@app.callback(
    
    Output(component_id='grafico-periodo',component_property='figure'),  
    Output(component_id='my_graph', component_property='figure'),
    Output(component_id="mapa",component_property="figure"),
    Input(component_id='botao-dropdown',component_property='value'),
    Input(component_id= 'selecione_causa', component_property='value'),
    Input(component_id="cor",component_property="value")   

)

def update_output(value,causas, cor):
    

#GRAFICO DE CAUSAS
    for causa in causas_single:

        if causa == causas:
            totales = calculo_causas(data_list,causa)
            
    fig2 = px.bar( x = dias_semana_single, y = totales)
    

    fig2.update_layout(plot_bgcolor="#272b30",paper_bgcolor="#272b30")
    fig2.update_xaxes(color='#aaaaaa')
    fig2.update_yaxes(color='#aaaaaa')

#GRAFICO DOS PERIODOS
    if value == 'Todos os períodos':

        fig = px.bar(nova_df, x='Dias da Semana', 
                            y = 'Quantidade', 
                            color='Periodo' , 
                            barmode='group',
                            )


        fig.update_layout(plot_bgcolor="#272b30",paper_bgcolor="#272b30",legend_title_font_color="#aaaaaa", font_color="#aaaaaa")
        fig.update_xaxes(color='#aaaaaa')
        fig.update_yaxes(color='#aaaaaa')
    else:

        conteudo_lista = filtro_periodo(value)

        valores = []        #Quebrando 
        semanas = []
        periodo = []
        
        for c in conteudo_lista :
            semanas.append(c[0])
            valores.append(c[1])
            periodo.append(c[2])

        df = pd.DataFrame((zip(semanas,valores,periodo)), columns = ['Dias da Semana', 'Quantidade', 'Periodo'])
        
        fig = px.bar(df, x='Dias da Semana',                                 #AS LINHAS QUE DEVERAO SER ANALISADAS SOMENTE SAO AS
                            y = 'Quantidade',                                #QUE SAO PERTENCENTES AOS VALORES INDICADO NO FILTRO
                            color='Periodo' , 
                            barmode='group',
                            )
                            
        fig.update_layout(plot_bgcolor="#272b30",paper_bgcolor="#272b30",legend_title_font_color="#aaaaaa", font_color="#aaaaaa")
        fig.update_xaxes(color='#aaaaaa')
        fig.update_yaxes(color='#aaaaaa')


#GRAFICO COROPLETICO

    df_mapa = newdata # cópia da base de dados para não alterar a original
    geojson = brasil_states # cópia do arquivo geojson para não alterar o original

    if cor == 'Mortos':  #se a opçao 'mortos' for selecionada, o mapa terá a seguinte aparência:
        fig_mapa = px.choropleth_mapbox(df_mapa, geojson=geojson, locations='UF', color = cor,
                            color_continuous_scale="Burg",                                                
                            center = {"lat": -13.6128, "lon": -48.5920}, 
                            range_color=(0, 530), 
                            zoom=3, 
                            opacity=0.4,   
                            hover_data={'Mortos': True, 'Feridos': False, 'Total': False, 'UF': True}   #define o que vai ser mostrado ao passar o mouse                 
    )   

    elif cor == 'Feridos': # Mas se a opçao 'feridos' for selecionada, o mapa terá a seguinte aparência:
        fig_mapa = px.choropleth_mapbox(df_mapa, geojson=geojson, locations='UF', color = cor,
                            color_continuous_scale="Burg",                                                
                            center = {"lat": -13.6128, "lon": -48.5920}, 
                            range_color=(0, 10000), 
                            zoom=3, 
                            opacity=0.4,   
                            hover_data={'Mortos': False, 'Feridos': True, 'Total': False, 'UF': True}   #define o que vai ser mostrado ao passar o mouse                 
    )   

    else:  # Por fim se a opçao 'total' for selecionada, o mapa terá a seguinte aparência:
        fig_mapa = px.choropleth_mapbox(df_mapa, geojson=geojson, locations='UF', color = cor,
                            color_continuous_scale="Burg",                                                
                            center = {"lat": -13.6128, "lon": -48.5920}, 
                            range_color=(0, 10000), 
                            zoom=3, 
                            opacity=0.4,   
                            hover_data={'Mortos': False, 'Feridos': False, 'Total': True, 'UF': True}    #define o que vai ser mostrado ao passar o mouse                
    )   




    fig_mapa.update_layout(
        mapbox_style="carto-darkmatter", #estilo
        autosize= True, #ajuste na tela
        margin=go.layout.Margin(l=300, r=300, t=0, b=0),
        plot_bgcolor="#272b30",
        paper_bgcolor="#272b30",
        legend_title_font_color="#aaaaaa",
         font_color="#aaaaaa"
    ) 

    return fig, fig2, fig_mapa


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run